# Validación e importación de los datos de interacción entre el usuario y los elementos <a class="anchor" id="top"></a>

En este cuaderno, elegirá un conjunto de datos y lo preparará para utilizarlo con Amazon Personalize.

1. [Introducción](#intro)
1. [Elija un conjunto de datos o un origen de datos](#source)
1. [Prepare los datos](#prepare)
1. [Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones](#group_dataset)
1. [Configurar un bucket de S3 y un rol de IAM](#bucket_role)
1. [Importar los datos de las interacciones](#import)

## Introducción <a class="anchor" id="intro"></a>

Por lo general, los algoritmos en Amazon Personalize (llamados recetas) buscan resolver diferentes tareas que se explican a continuación:

1. **User Personalization (Personalización del usuario)**: nueva versión que admite TODOS los flujos de trabajo de HRNN/necesidades de personalización del usuario, será lo que utilicemos aquí.
1. **HRNN & HRNN-Metadata**: recomienda elementos basados en las interacciones previas del usuario con los elementos.
1. **HRNN-Coldstart**: recomienda nuevos elementos para los que aún no se dispone de datos de interacción.
1. **Personalized-Ranking**: toma una recopilación de elementos y, luego, los ubica en un posible orden de interés mediante un enfoque similar al de una HRNN.
1. **SIMS (elementos similares)**: en base a un elemento, recomienda otros elementos con los que también interactúan los usuarios.
1. **Popularity-Count**: recomienda los elementos más populares, si la HRNN o los metadatos de la HRNN no ofrecen una respuesta. Esta es la devolución predeterminada.

Sin importar cuál sea el caso de uso, todos los algoritmos comparten una base de aprendizaje sobre los datos de interacción entre el usuario y los elementos, la cual se define según 3 atributos principales:

1. **UserID**: el usuario que interactuó
1. **ItemID**: el elemento con el que interactuó el usuario
1. **Timestamp (Marca temporal)**: la hora a la que ocurrió la interacción

También admitimos los tipos de eventos y valores de eventos definidos por lo siguiente:

1. **Tipo de evento**: etiquetado de eventos por categoría (navegación, compra, clasificación, etc.).
1. **Valor de evento**: valor que corresponde al tipo de evento acontecido. En términos generales, buscamos valores normalizados entre 0 y 1 entre los tipos de eventos. Por ejemplo, si existen tres fases para completar una transacción (seleccionar, agregar al carrito y comprar), entonces hay un valor de evento para cada fase, como 0.33, 0.66 y 1.0, respectivamente.

Los campos para el tipo de evento y valor de evento son datos adicionales que se pueden utilizar para filtrar los datos enviados para el entrenamiento del modelo de personalización. En este ejercicio en particular no tendremos un tipo de evento ni un tipo de valor. 

## Elija un conjunto de datos o un origen de datos <a class="anchor" id="source"></a>
[Regresar al principio](#top)

Como ya lo mencionamos, los datos de la interacción del usuario con los elementos son cruciales para comenzar con el servicio. Esto significa que necesitamos buscar los casos de uso que generan ese tipo de datos, algunos ejemplos comunes son los siguientes:

1. Aplicaciones de video bajo demanda
1. Plataformas de comercio electrónico
1. Agregadores/plataformas de redes sociales

Existen algunas pautas para evaluar un problema que sea adecuado para Personalize. Como punto inicial, recomendamos los valores que figuran a continuación, aunque los [límites oficiales](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) son un poco más bajos.

* Usuarios autenticados
* Al menos 50 usuarios exclusivos
* Al menos 100 elementos exclusivos
* Al menos 2 docenas de interacciones para cada usuario 

La mayoría de las veces esto es fácil de lograr, y si no llega a esta cantidad en alguna categoría, puede compensarlo con una mayor cantidad en otra categoría.

En general, Personalize no enviará los datos en forma perfecta y deberá realizar algunas modificaciones para que la estructura sea la correcta. Este cuaderno tiene como fin guiarlo en este proceso. 

Para comenzar, utilizaremos el último conjunto de datos de MovieLens, que cuenta con más de 25 millones de interacciones y una rica recopilación de metadatos para los elementos. También hay una versión más pequeña de este conjunto de datos, que se puede utilizar para acortar los tiempos de entrenamiento, sin dejar de incorporar las mismas capacidades que el conjunto de datos completo. Establezca USE_FULL_MOVIELENS en True (Verdadero) para utilizar el conjuntos de datos completo.

In [1]:
USE_FULL_MOVIELENS = False

Primero, descargará el conjunto de datos y lo extraerá en una nueva carpeta por medio del código que aparece a continuación.

In [2]:
data_dir = "poc_data"
!mkdir $data_dir

if not USE_FULL_MOVIELENS:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    !cd $data_dir && unzip ml-latest-small.zip
    dataset_dir = data_dir + "/ml-latest-small/"
else:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !cd $data_dir && unzip ml-25m.zip
    dataset_dir = data_dir + "/ml-25m/"

--2020-09-16 21:10:35--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.77MB/s    in 0.2s    

2020-09-16 21:10:35 (3.77 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


Observe los archivos de datos que ha descargado.

In [3]:
!ls $dataset_dir

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


En este momento no hay mucha información, salvo que tenemos unos cuantos archivos CSV y un archivo README. A continuación, generaremos el archivo README para obtener más información.

In [4]:
!pygmentize $dataset_dir/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

En el archivo README, vemos que hay un archivo `ratings.csv` que debería funcionar como un proxy para nuestros datos de las interacciones. En definitiva, calificar una película sin dudas es una forma de interactuar con ella. El conjunto de datos también tiene algo de información sobre el género, como algunos datos del genoma de la película. En esta POC, nos centraremos en las interacciones y los datos sobre el género.


## Prepare los datos <a class="anchor" id="prepare"></a>
[Regresar al principio](#top)

Lo que debe hacer a continuación es cargar los datos y confirmar que los datos se encuentran en buen estado. Luego, debe guardarlos en un archivo CSV donde estén listos para su uso con Amazon Personalize.

Para comenzar, importe una recopilación de bibliotecas de Python que se utilicen normalmente en la ciencia de datos.

In [5]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

Luego, abra el archivo de datos y observe varias de las primeras filas.

In [6]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
original_data.shape

(100836, 4)

In [8]:
original_data.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


Esto muestra que tenemos un buen rango de valores para `userId` y `movieId`. Luego, siempre es una buena idea confirmar el formato de los datos.

In [9]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [10]:
original_data.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

A partir de esto, se puede ver que hay un total de 25 000 095 entradas en el conjunto de datos completo y 100 836 entradas en el pequeño, con 4 columnas, y que cada celda almacenada tiene formato int64, con la excepción de la calificación cuyo formato es float64.

El formato int64 es claramente adecuado para `userId` y `movieId`. Sin embargo, necesitamos examinar más a fondo para comprender las marcas temporales en los datos. Para utilizar Amazon Personalize, necesita guardar las marcas temporales en el formato [Unix Epoch (época de Unix)](https://en.wikipedia.org/wiki/Unix_time).

Actualmente, los valores de las marcas temporales no son legibles para el ser humano. Así que tomemos un valor de marca temporal arbitrario y resolvamos cómo interpretarlo.

Haga una comprobación de validez rápida en el conjunto de datos transformado mediante la elección de una marca temporal arbitraria y su transformación a un formato legible para humanos.

In [11]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

964982681.0
2000-07-30 18:44:41


Estos datos son coherentes como una marca temporal, para que podamos continuar formateando el resto de los datos. Recuerde que los datos que necesitamos son datos de interacción entre el usuario y los elementos, que serían `userId`, `movieId` y `timestamp` en este caso. Nuestro conjunto de datos tiene una columna adicional, `rating`, que puede eliminarse del conjunto de datos después de que la hayamos aprovechado para centrarnos en las interacciones positivas.

Como se trata de un conjunto de datos de calificación de películas con opiniones explícitas, que incluye películas con calificaciones del 1 al 5, queremos incluir solo las movimientos que "gustaron" a los usuarios, y simular un conjunto de datos implícito que se asemeje más a los datos que recopilaría una plataforma VOD. Para ello filtraremos todas las interacciones entre 2 y 5, y crearemos un EVENT_Type de "clics" y un EVENT_Type de "vistas". A continuación, asignaremos todas las películas calificadas con 2 o más como "clics" y las películas con una clasificación mayor de 4 "clics" y "vistas".

Tenga en cuenta que esto es para corresponder con los eventos que estamos modelando, para un conjunto de datos reales que en verdad modelaría en función de las opiniones implícitas, como los clics y las vistas, o de las opiniones explícitas, como las calificaciones, los “Me gusta”, etc.

In [12]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'
watched_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,1,964982703,watch
1,1,3,964981247,watch
2,1,6,964982224,watch
3,1,47,964983815,watch
4,1,50,964982931,watch


In [13]:
clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'
clicked_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,1,964982703,click
1,1,3,964981247,click
2,1,6,964982224,click
3,1,47,964983815,click
4,1,50,964982931,click


In [14]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [15]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158371 entries, 66679 to 81092
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   userId      158371 non-null  int64 
 1   movieId     158371 non-null  int64 
 2   timestamp   158371 non-null  int64 
 3   EVENT_TYPE  158371 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.0+ MB


Veamos el aspecto del conjunto de datos nuevo. 

In [16]:
interactions_df.describe()

,userId,movieId,timestamp
count,158371.000000,158371.000000,1.583710e+05
mean,323.940804,19869.485146,1.211627e+09
std,182.195975,35809.058185,2.137376e+08
min,1.000000,1.000000,8.281246e+08
25%,175.000000,1203.000000,1.031072e+09
50%,325.000000,3033.000000,1.193289e+09
75%,475.000000,8528.000000,1.435998e+09
max,610.000000,193609.000000,1.537799e+09


Luego de manipular los datos, siempre confirme si el formato de datos ha sido modificado.

In [17]:
interactions_df.dtypes

userId         int64
movieId        int64
timestamp      int64
EVENT_TYPE    object
dtype: object

 Amazon Personalize tiene nombres de columnas predeterminados para los usuarios, los elementos y las marcas temporales. Estos nombres de columnas predeterminados son `USER_ID`, `ITEM_ID` Y `TIMESTAMP`. Por lo que la última modificación del conjunto de datos es reemplazar la columna de encabezados existente por los encabezados predeterminados.

In [18]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

¡Eso es todo! A esta altura, los datos están listos y solo necesitamos guardarlos como un archivo CSV.

In [19]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones <a class="anchor" id="group_dataset"></a>
[Regresar al principio](#top)

El nivel más alto de aislamiento y abstracción con Amazon Personalize es un *grupo de conjuntos de datos*. La información almacenada dentro de uno de estos grupos de conjunto de datos no afecta ninguno de los demás grupos de conjuntos de datos o modelos creados de uno de ellos; están completamente aislados. Esto permite ejecutar muchos experimentos y es parte de lo que hacemos para mantener a los modelos de forma privada y completamente entrenados en sus datos. 

Antes de importar los datos que preparó antes, debe haber un grupo de conjuntos de datos y un conjunto de datos agregados a este que maneje las interacciones.

Los grupos de conjuntos de datos pueden almacenar los siguientes tipos de información:

* Interacciones entre el usuario y el elemento
* Flujo de eventos (interacciones en tiempo real)
* Metadatos del usuario
* Metadatos del elemento

Antes de crear el grupo de conjuntos de datos y el conjunto de datos para nuestros datos de interacción, validemos que su entorno se pueda comunicar exitosamente con Amazon Personalize.

In [20]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Crear el grupo de conjuntos de datos

La siguiente celda creará un nuevo grupo de conjuntos de datos con el nombre `personalize-poc-lastfm`.

In [21]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-poc-movielens"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:136455442858:dataset-group/personalize-poc-movielens",
  "ResponseMetadata": {
    "RequestId": "9e2a4940-beae-4e2e-819e-d1f08d80aa20",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 21:11:35 GMT",
      "x-amzn-requestid": "9e2a4940-beae-4e2e-819e-d1f08d80aa20",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, este debe estar activo. Esto puede llevar uno o dos minutos. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del grupo de conjuntos de datos a cada segundo, hasta por un máximo de 3 horas.

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Ahora que tiene un grupo de conjuntos de datos, puede crear un conjunto de datos para los datos de interacción.

### Crear el conjunto de datos

Primero, defina un esquema para indicar a Amazon Personalize el tipo de conjunto de datos que está cargando. Existen varias palabras clave reservadas y obligatorias, necesarias para el esquema, según el tipo de conjunto de datos. Para obtener más información, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Aquí, creará un esquema para datos de interacciones, para lo que se necesitan los campos `USER_ID`, `ITEM_ID` y `TIMESTAMP`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.

In [23]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:136455442858:schema/personalize-poc-movielens-interactions",
  "ResponseMetadata": {
    "RequestId": "9f851524-b3c1-414e-a969-c1581055302b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 21:12:37 GMT",
      "x-amzn-requestid": "9f851524-b3c1-414e-a969-c1581055302b",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Con un esquema creado, puede crear un conjunto de datos dentro del grupo de conjuntos de datos. Nota: Esto aún no carga los datos. Esto sucederá unos pasos más adelante.

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:136455442858:dataset/personalize-poc-movielens/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "a651b8a9-2989-4582-a407-7ddd640827d1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 21:12:38 GMT",
      "x-amzn-requestid": "a651b8a9-2989-4582-a407-7ddd640827d1",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>
[Regresar al principio](#top)

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

In [25]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Los nombres de los buckets de Amazon S3 son exclusivos a nivel global. Para crear un nombre de bucket exclusivo, el siguiente código agregará la cadena `personalizepocvod` a su número de cuenta de AWS. Luego, crea un bucket con este nombre en la región descubierta en la celda anterior.

In [28]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizepocvod"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

136455442858-us-east-1-personalizepocvod


{'ResponseMetadata': {'RequestId': 'BD15606D3F8E215F',
  'HostId': 'Kc9qItw32VyAXhDKY2GFyCEzpLkUMly/+ukG/ZkiW9b/JVwymZ/3hA3m9Z4UjgOOodS6//T9bVo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Kc9qItw32VyAXhDKY2GFyCEzpLkUMly/+ukG/ZkiW9b/JVwymZ/3hA3m9Z4UjgOOodS6//T9bVo=',
   'x-amz-request-id': 'BD15606D3F8E215F',
   'date': 'Wed, 16 Sep 2020 21:18:12 GMT',
   'location': '/136455442858-us-east-1-personalizepocvod',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/136455442858-us-east-1-personalizepocvod'}

### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos. 

In [29]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

In [30]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '2F6317C3D577280F',
  'HostId': '8h3eX6vtbGYKSNX0VmLgv+gY1/AMVeN+iZ8rvIwzCvuQNv3o6LsvSx+HD9WrpYcAySGQCBBwn8c=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '8h3eX6vtbGYKSNX0VmLgv+gY1/AMVeN+iZ8rvIwzCvuQNv3o6LsvSx+HD9WrpYcAySGQCBBwn8c=',
   'x-amz-request-id': '2F6317C3D577280F',
   'date': 'Wed, 16 Sep 2020 21:18:40 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Crear un rol de IAM

Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Creemos un rol de IAM y adjuntemos las políticas necesarias a este. El siguiente código adjunta políticas muy permisivas. Utilice políticas más restrictivas para cualquier aplicación en producción.

In [31]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::136455442858:role/PersonalizeRolePOC


## Importar los datos de las interacciones <a class="anchor" id="import"></a>
[Regresar al principio](#top)

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [32]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:136455442858:dataset-import-job/personalize-poc-import1",
  "ResponseMetadata": {
    "RequestId": "5bd19035-90da-42e7-902c-9df1fdcdaa49",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 21:19:43 GMT",
      "x-amzn-requestid": "5bd19035-90da-42e7-902c-9df1fdcdaa49",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, el trabajo de importación debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del trabajo de importación a cada segundo, por hasta un máximo de 6 horas.

La importación de los datos puede llevar algo de tiempo, según el tamaño del conjunto de datos. En este taller, el trabajo de importación debería tomar alrededor de 15 minutos.

In [33]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 108 ms, sys: 15.2 ms, total: 123 ms
Wall time: 17min 1s


Cuando la importación del conjunto de datos está activa, está listo para comenzar a crear modelos con SIMS, Personalized-Ranking, Popularity-Count y User Personalization. Este proceso continuará en otros cuadernos. Ejecute la celda a continuación antes de pasar a almacenar algunos valores para su uso en los siguientes cuadernos.

In [34]:
%store USE_FULL_MOVIELENS
%store dataset_dir
%store interactions_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store data_dir
%store region
%store interaction_schema_arn

Stored 'USE_FULL_MOVIELENS' (bool)
Stored 'dataset_dir' (str)
Stored 'interactions_dataset_arn' (str)
Stored 'dataset_group_arn' (str)
Stored 'bucket_name' (str)
Stored 'role_arn' (str)
Stored 'role_name' (str)
Stored 'data_dir' (str)
Stored 'region' (str)
Stored 'interaction_schema_arn' (str)
